In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:


# Path to SALICON raw data
pathToImagesTrain = '/content/drive/MyDrive/IVP_Project_ISP/images/train/*.jpg'
pathToImagesVal = '/content/drive/MyDrive/IVP_Project_ISP/images/val/*.jpg'
pathToImagesTest = '/content/drive/MyDrive/IVP_Project_ISP/images/test/*.jpg'

pathToMapsTrain = '/content/drive/MyDrive/IVP_Project_ISP/maps/train/*.png'
pathToMapsVal = '/content/drive/MyDrive/IVP_Project_ISP/maps/val/*.png'

# Path to processed data
pathToResizedImagesTrain = '/content/drive/MyDrive/IVP_ISP/Preprocessed_data/images256x192_train/'
pathToResizedMapsTrain = '/content/drive/MyDrive/IVP_ISP/Preprocessed_data/maps256x192_train/'

pathToResizedImagesVal = '/content/drive/MyDrive/IVP_ISP/Preprocessed_data/images256x192_val/'
pathToResizedMapsVal = '/content/drive/MyDrive/IVP_ISP/Preprocessed_data/maps256x192_val/'

pathToResizedImagesTest = '/content/drive/MyDrive/IVP_ISP/Preprocessed_data/images256x192_test/'

INPUT_SIZE = (256, 192)

In [ ]:
#####################################################
''' Data Loader '''
#####################################################

import glob
import os
import numpy as np
import cv2
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


class DataLoader(object):

    def __init__(self, batch_size = 5):
        #reading data list
        self.list_img = list_img = [k.split('/')[-1].split('.')[0] for k in sorted(glob.glob(pathToResizedImagesTrain+'*.png'))]
        self.batch_size = batch_size
        self.size = len(self.list_img)
        self.cursor = 0
        self.num_batches = self.size / batch_size

    def get_batch(self): # Returns 
        if self.cursor + self.batch_size > self.size:
            self.cursor = 0
            np.random.shuffle(self.list_img)
            
        img = torch.zeros(self.batch_size, 3, 192, 256)
        sal_map = torch.zeros(self.batch_size, 1, 192, 256)
        
        #to_tensor = transform.Compose(transforms.ToTensor() # Transforms 0-255 numbers to 0 - 1.0.
        to_tensor = transforms.ToTensor() # Transforms 0-255 numbers to 0 - 1.0.

        for idx in range(self.batch_size):
            curr_file = self.list_img[self.cursor]
            full_img_path = os.path.join(pathToResizedImagesTrain, curr_file + '.png')
            full_map_path = os.path.join(pathToResizedMapsTrain, curr_file + '.png')
            self.cursor += 1
            inputimage = cv2.imread(full_img_path) # (192,256,3)
            img[idx] = to_tensor(inputimage)
            
            saliencyimage = cv2.imread(full_map_path,0)
            saliencyimage = np.expand_dims(saliencyimage,axis=2)
            sal_map[idx] = to_tensor(saliencyimage)
            
        return (img,sal_map)

In [ ]:
#DL = DataLoader(5)

In [ ]:
#img, map = DL.get_batch()

In [ ]:
'''plt.subplot(121)
plt.imshow(img[4,0,:,:])
plt.subplot(122)
plt.imshow(map[4,0,:,:])'''


'plt.subplot(121)\nplt.imshow(img[4,0,:,:])\nplt.subplot(122)\nplt.imshow(map[4,0,:,:])'

In [ ]:
import torch.nn as nn

def conv2d(in_channels, out_channels, kernel_size = 3, padding = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size = kernel_size, padding = padding)

def deconv2d(in_channels, out_channels, kernel_size = 3, padding = 1):
    return nn.ConvTranspose2d(in_channels, out_channels, kernel_size = kernel_size, padding = padding)

def relu(inplace = True): # Change to True?
    return nn.ReLU(inplace)

def maxpool2d():
    return nn.MaxPool2d(2)

In [ ]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import math
from torch.autograd import Variable

def make_conv_layers(cfg):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [maxpool2d()]
        else:
            conv = conv2d(in_channels, v)
            layers += [conv, relu(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


def make_deconv_layers(cfg):
    layers = []
    in_channels = 512
    for v in cfg:
        if v == 'U':
            layers += [nn.Upsample(scale_factor=2)]
        else:
            deconv = deconv2d(in_channels, v)
            layers += [deconv]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512],
    'D': [512, 512, 512, 'U', 512, 512, 512, 'U', 256, 256, 256, 'U', 128, 128, 'U', 64, 64]
}

def encoder():
    return make_conv_layers(cfg['E'])

def decoder():
    return make_deconv_layers(cfg['D'])

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = encoder()
        self.decoder = decoder()
        self.mymodules = nn.ModuleList([
            deconv2d(64,1,kernel_size=1, padding = 0),
            nn.Sigmoid()
        ])

    def forward(self,x): #
        #print('Input x', x.size())
        x = self.encoder(x)
        #print('After encoder = ', x.size())
        x = self.decoder(x)
        #print('After decoder = ', x.size())
        x = self.mymodules[0](x)
        x = self.mymodules[1](x)
        #print('Final size = ', x.size())
        return x


In [ ]:
import torch.nn as nn
import torch
import math
from torch.autograd import Variable

#os.environ['CUDA_VISIBLE_DEVICES'] = '2'

class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.convs = nn.Sequential( # [-1, 4, 256,192]
                conv2d(4,3,kernel_size=1),
                relu(),
                conv2d(3,32,kernel_size=3), #[-1, 32, 256, 192]
                relu(),
                maxpool2d(),
                conv2d(32,64,kernel_size=3), #[-1, 64, 128, 96]
                relu(),
                conv2d(64,64,kernel_size=3), #[-1, 64, 128, 96]
                relu(),
                maxpool2d(), #[-1,64,64,48]
                conv2d(64,64,kernel_size=3), #[-1,64,64,48]
                relu(),
                conv2d(64,64,kernel_size=3),
                relu(),
                maxpool2d(), #[-1,64,32,24]
        )
        self.mymodules = nn.ModuleList([
            nn.Sequential(nn.Linear(64*32*24, 100), nn.Tanh()),
            nn.Sequential(nn.Linear(100,2), nn.Tanh()),
            nn.Sequential(nn.Linear(2,1), nn.Sigmoid())
        ])
        #self._initialize_weights()

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.mymodules[0](x)
        x = self.mymodules[1](x)
        x = self.mymodules[2](x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                #print(m.weight.data.shape)
                #print('old conv2d layer!')
                #print(m.weight.data.min())
                #print(m.weight.data.max())
                m.weight.data.normal_(0, math.sqrt(2. / n))
                #print('new conv2d layer!')
                #print(m.weight.data.min())
                #print(m.weight.data.max())
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
                

In [ ]:
import cv2
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

from PIL import Image

def to_variable(x,requires_grad=True):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x,requires_grad)

def show(img):
    #print(img.shape)
    pilTrans = transforms.ToPILImage()
    pilImg = pilTrans(img)
    s = np.array(pilImg)
    plt.figure()
    plt.imshow(s)
    
def show_gray(img):
    print(img.shape)
    pilTrans = transforms.ToPILImage()
    pilImg = pilTrans(img)
    s = np.array(pilImg)
    plt.figure()
    plt.imshow(s)
    
def save_gray(img, path):
    pilTrans = transforms.ToPILImage()
    pilImg = pilTrans(img)
    print('Image saved to ', path)
    pilImg.save(path)

    
def predict(model, img, epoch, path):
    to_tensor = transforms.ToTensor() # Transforms 0-255 numbers to 0 - 1.0.
    im = to_tensor(img)
    #show(im)
    inp = to_variable(im.unsqueeze(0), False)
    #print(inp.size())
    out = model(inp)
    map_out = out.cpu().data.squeeze(0)
    #show_gray(map_out)
    
    new_path = path + str(epoch) + ".png"
    save_gray(map_out, new_path)
    

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Logger(object):

    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

# Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                    height=img.shape[0],
                    width=img.shape[1])
# Create a Summary value
            img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, i), image=img_sum))

# Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)

    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

# Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

# Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

# Drop the start of the first bin
        bin_edges = bin_edges[1:]

# Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

# Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import numpy as np
import time
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
from tqdm import tqdm
from torch.autograd import Variable
#from logger import Logger

#from data_loader import DataLoader
#from discriminator import Discriminator
#from generator import Generator
#from utils import *

#logger = Logger()
batch_size = 16
lr = 0.0003

discriminator = Discriminator()
generator = Generator()
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()
loss_function = nn.BCELoss()

d_optim = torch.optim.Adagrad(discriminator.parameters(), lr=lr)
g_optim = torch.optim.Adagrad(generator.parameters(), lr=lr)

num_epoch = 100
dataloader = DataLoader(batch_size)
num_batch = dataloader.num_batches# length of data / batch_size
print(num_batch)

def to_variable(x,requires_grad=True):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x,requires_grad)

counter = 0
start_time = time.time()
DIR_TO_SAVE = "/content/drive/MyDrive/IVP_ISP/Save_directory/"
if not os.path.exists(DIR_TO_SAVE):
    os.makedirs(DIR_TO_SAVE)
validation_sample = cv2.imread("/content/drive/MyDrive/IVP_ISP/Preprocessed_data/images256x192_val/1025.png")

for current_epoch in tqdm(range(1,num_epoch+1)):
    n_updates = 1
    
    d_cost_avg = 0
    g_cost_avg = 0
    for idx in range(int(num_batch)):
            
        (batch_img, batch_map) = dataloader.get_batch()
        batch_img = to_variable(batch_img,requires_grad=False)
        batch_map = to_variable(batch_map,requires_grad=False)
        real_labels = to_variable(torch.FloatTensor(np.ones(batch_size, dtype = float)),requires_grad=False)
        fake_labels = to_variable(torch.FloatTensor(np.zeros(batch_size, dtype = float)),requires_grad=False)
        
        if n_updates % 2 == 1:
            #print('Training Discriminator...')
            #discriminator.zero_grad()
            d_optim.zero_grad()
            inp_d = torch.cat((batch_img,batch_map),1)            
            #print(inp_d.size())
            outputs = discriminator(inp_d).squeeze()
            d_real_loss = loss_function(outputs,real_labels)
            #print('D_real_loss = ', d_real_loss.data[0])
            
            #print(outputs)
            real_score = outputs.data.mean()

#            fake_map = generator(batch_img)
#            inp_d = torch.cat((batch_img,fake_map),1)
#            outputs = discriminator(inp_d)
#            d_fake_loss = loss_function(outputs, fake_labels)
#            print('D_fake_loss = ', d_fake_loss.data[0])
            d_loss = torch.sum(torch.log(outputs))
            d_cost_avg += d_loss.data
            
            d_loss.backward()
            d_loss.register_hook(print)
            d_optim.step()
            info = {
                 'd_loss' : d_loss.data,
                 'real_score_mean' : real_score,
            }
            #for tag,value in info.items():
                #logger.scalar_summary(tag, value, counter)
        else:
            #print('Training Generator...')
            #generator.zero_grad()
            g_optim.zero_grad()
            fake_map = generator(batch_img)
            inp_d = torch.cat((batch_img,fake_map),1)
            outputs = discriminator(inp_d)
            fake_score = outputs.data.mean()

            g_gen_loss = loss_function(fake_map,batch_map)
            g_dis_loss = -torch.log(outputs)
            alpha = 0.05
            g_loss = torch.sum(g_dis_loss + alpha * g_gen_loss)

            g_cost_avg += g_loss.data
            
            g_loss.backward()
            g_optim.step()
            info = {
                  'g_loss' : g_loss.data,
                  'fake_score_mean' : fake_score,
            }
            #for tag,value in info.items():
                #logger.scalar_summary(tag, value, counter)

        n_updates += 1

        if (idx+1)%100 == 0:
            print("Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, D(x): %2.f, D(G(x)): %.2f, time: %4.4f"
		        % (current_epoch, num_epoch, idx+1, num_batch, d_loss.data, g_loss.data,
		        real_score, fake_score, time.time()-start_time))
        counter += 1
    d_cost_avg /= num_batch
    g_cost_avg /= num_batch
        
    # Save weights every 10 epoch
    if (current_epoch + 1) % 10 == 0:
        print('Epoch:', current_epoch, ' train_loss->', (d_cost_avg, g_cost_avg))
        torch.save(generator.state_dict(), '/content/drive/MyDrive/IVP_ISP/Model_saved/Generator/generator.h5')
        torch.save(discriminator.state_dict(), '/content/drive/MyDrive/IVP_ISP/Model_saved/Discriminator/discriminator.h5')
    predict(generator, validation_sample, current_epoch, DIR_TO_SAVE)
torch.save(generator.state_dict(), '/content/drive/MyDrive/IVP_ISP/Model_saved/Improved_with_100_epochs/generator.h5')
torch.save(discriminator.state_dict(), '/content/drive/MyDrive/IVP_ISP/Model_saved/Improved_with_100_epochs/discriminator.h5')
print('Done')

625.0


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Step[100/625], d_loss: -16.7756, g_loss: 17.0573, D(x):  0, D(G(x)): 0.35, time: 1623.7799
Epoch [1/100], Step[200/625], d_loss: -16.8351, g_loss: 17.1420, D(x):  0, D(G(x)): 0.35, time: 3260.6592
Epoch [1/100], Step[300/625], d_loss: -16.8801, g_loss: 17.1840, D(x):  0, D(G(x)): 0.35, time: 4958.9332
Epoch [1/100], Step[400/625], d_loss: -16.9179, g_loss: 17.1850, D(x):  0, D(G(x)): 0.35, time: 6575.6669
Epoch [1/100], Step[500/625], d_loss: -16.9511, g_loss: 17.2095, D(x):  0, D(G(x)): 0.35, time: 8597.4253
Epoch [1/100], Step[600/625], d_loss: -16.9812, g_loss: 17.1913, D(x):  0, D(G(x)): 0.35, time: 10726.4191
Image saved to  /content/drive/MyDrive/IVP_ISP/Save_directory/1.png


  1%|          | 1/100 [3:07:35<309:31:42, 11255.58s/it]

Epoch [2/100], Step[100/625], d_loss: -17.0156, g_loss: 17.2339, D(x):  0, D(G(x)): 0.35, time: 11289.3132
Epoch [2/100], Step[200/625], d_loss: -17.0408, g_loss: 17.2744, D(x):  0, D(G(x)): 0.34, time: 11323.0647
Epoch [2/100], Step[300/625], d_loss: -17.0645, g_loss: 17.3134, D(x):  0, D(G(x)): 0.34, time: 11356.8625
Epoch [2/100], Step[400/625], d_loss: -17.0869, g_loss: 17.3113, D(x):  0, D(G(x)): 0.34, time: 11390.6080
Epoch [2/100], Step[500/625], d_loss: -17.1082, g_loss: 17.3310, D(x):  0, D(G(x)): 0.34, time: 11424.3889
Epoch [2/100], Step[600/625], d_loss: -17.1286, g_loss: 17.3479, D(x):  0, D(G(x)): 0.34, time: 11458.2767
Image saved to  /content/drive/MyDrive/IVP_ISP/Save_directory/2.png


...

 99%|█████████▉| 99/100 [8:54:46<03:33, 213.22s/it]

Image saved to  /content/drive/MyDrive/IVP_ISP/Save_directory/99.png
Epoch [100/100], Step[100/625], d_loss: -20.2044, g_loss: 20.3658, D(x):  0, D(G(x)): 0.28, time: 32120.3886
Epoch [100/100], Step[200/625], d_loss: -20.2074, g_loss: 20.4253, D(x):  0, D(G(x)): 0.28, time: 32154.6606
Epoch [100/100], Step[300/625], d_loss: -20.2104, g_loss: 20.3797, D(x):  0, D(G(x)): 0.28, time: 32188.7989
Epoch [100/100], Step[400/625], d_loss: -20.2134, g_loss: 20.3953, D(x):  0, D(G(x)): 0.28, time: 32222.8319
Epoch [100/100], Step[500/625], d_loss: -20.2164, g_loss: 20.3936, D(x):  0, D(G(x)): 0.28, time: 32256.8297
Epoch [100/100], Step[600/625], d_loss: -20.2193, g_loss: 20.4172, D(x):  0, D(G(x)): 0.28, time: 32291.0886


100%|██████████| 100/100 [8:58:19<00:00, 322.99s/it]

Image saved to  /content/drive/MyDrive/IVP_ISP/Save_directory/100.png


Done
